In [1]:
from py2neo import Node, Relationship
from py2neo import Graph
from py2neo.ogm import GraphObject, RelatedFrom, RelatedTo
from py2neo.ogm import Property

import json

In [2]:
# build connection between py2neo and neo4j
graph = Graph(password='aaduser')
print('done~')

done~


In [3]:
print(len(graph.nodes))
graph.nodes[838485]

838488


(_838485:Officer {countries: 'Taiwan', country_codes: 'TWN', name: 'NIU, Fu-Tsang', node_id: '71202', sourceID: 'Offshore Leaks', valid_until: 'The Offshore Leaks data is current through 2010'})

In [4]:
# build 2 classes for node
class Officer(GraphObject):
    name = Property()
    node_id = Property()
    countries = Property()

    # registered_address = RelatedTo(Address)
    officer_of = RelatedTo("Entity")
    shareholder_of = RelatedTo("Entity")
    director_of = RelatedTo("Entity")
    # maybe not work for other leak data
    alternate_director_of = RelatedTo("Entity")
    # intermediary_of = RelatedTo()

class Entity(GraphObject):
    name = Property()
    node_id = Property()
    
    officers = RelatedFrom("Officer","OFFICER_OF")
    shareholders = RelatedFrom("Officer","SHAREHOLDER_OF")
    directors = RelatedFrom("Officer","ALTERNATE_DIRECTOR_OF")



In [5]:
# build the empty dictionary for store the data from neo4j
the_dict = {"elements":{"nodes":[],"edges":[]}, "style":{}, "layout":{ "name": "grid"}}

In [9]:


# search a name
# a_name = "Barry Neil Mitchell"
a_name = "Thierry Fernand Martino"

# get the subgraph from neo4j base on the name given
node_out = Officer.match(graph).where(name=a_name).first()


In [10]:
# the_dict["elements"]["nodes"]

# build empty list for nodes and edges
node_list = []
edge_list = []

# add the core node to the node list
node_list.append({"data":{"id":node_out.node_id,"name":node_out.name,"country":node_out.countries,"label":"Officer"}})

# add the nodes and edges from the nodes of shareholder_of
for x in node_out.shareholder_of:
    node_list.append({"data":{"id":x.node_id,"name":x.name,"label":"Entity"}})
    edge_list.append({"data":{"source":node_out.node_id,"target":x.node_id,"relationship":"OFFICER_OF"}})

# add the nodes and edges from the nodes of alternate_director_of
for x in node_out.alternate_director_of:
    node_list.append({"data":{"id":x.node_id,"name":x.name,"label":"Entity"}})
    edge_list.append({"data":{"source":node_out.node_id,"target":x.node_id,"relationship":"ALTERNATE_DIRECTOR_OF"}})

# add the node list and edge list to the main dictionary ( the_dict )
the_dict["elements"]["nodes"] = node_list
the_dict["elements"]["edges"] = edge_list

# print(the_dict)
json.dumps(the_dict) # trans to json format

# write json to file
with open('test_json.json','w') as json_file:
    json.dump(the_dict,json_file)
print('writed the data to test_json.json.')

writed the data to test_json.json.


In [8]:
a_all = Officer.match(graph).where("_.name =~ 'Thierry.*'").limit(50)
for a in a_all:
    print(a.name)
    for share_of in a.shareholder_of:
        print('---' + share_of.name)

Thierry Fernand Martino
---Always Rock Ltd.
Thierry Mooij
---JESS MARKETING SERVICES LLC
Thierry LEVERRIER
---IRVINE PORTFOLIO S.A.
Thierry Semichon-Hobaïca
---ANDSELL S.A.
Thierry ISAIA
---ALPHA GLOBAL INDUSTRIES LTD.
Thierry ISAIA
---Jet Ambulance International Limited
Thierry Dollat
---HTD LIMITED
Thierry Hugnin
Thierry Amaud
---Pepsoftware International Limited
Thierry Renard
---Infoshare Ltd.
---Asia-InfoShare Ltd.


In [10]:
one_man = Officer.match(graph).where(name="Barry Neil Mitchell").first()
print(one_man.name)
print(one_man.__node__)

Barry Neil Mitchell
(_774656:Officer {countries: 'British Virgin Islands', country_codes: 'VGB', name: 'Barry Neil Mitchell', node_id: '118387', sourceID: 'Offshore Leaks', valid_until: 'The Offshore Leaks data is current through 2010'})


In [11]:
test_node = Officer.match(graph,"Barry Neil Mitchell")
print(type(test_node))
for node in test_node:
    print(type(node))
{("name:"+a.name) for a in Officer.match(graph).limit(5)}

<class 'py2neo.ogm.OfficerMatch'>


{'name:GREGORY JOHN SOLOMON',
 'name:HO THUY NGA',
 'name:KIM SOO IN',
 'name:MATSUDA MASUMI',
 'name:Tian Yuan'}

In [7]:
{("name:"+a.name) for a in Person.match(graph).limit(5)}

NameError: name 'Person' is not defined

In [11]:
top10 = [(a.tag_line, a.released, a.title, a.__primaryvalue__) for a in Movie.match(graph,primary_value="Top Gun").limit(10)]
print(top10)

[('I feel the need, the need for speed.', 1986, 'Top Gun', 'Top Gun')]


In [14]:
person = Person.match(graph).where(born=1962).first()
print(person)
print(person.name)
print(person.born)

<Person __id__=839435>
Tom Cruise
1962


In [51]:
for act in person.acted_in:
    print(act.title)

A Few Good Men
Top Gun
Jerry Maguire


In [15]:
person2 = Person.match(graph).where(born=1962)
for per in person2:
    print(per.name)
    for act in per.acted_in:
        print('---'+act.title)

Tom Cruise
---A Few Good Men
---Top Gun
---Jerry Maguire
Demi Moore
---A Few Good Men
Anthony Edwards
---Top Gun
Kelly Preston
---Jerry Maguire
Rosie O'Donnell
---A League of Their Own
---Sleepless in Seattle


In [16]:
person_list = list(Person.match(graph).where("_.name =~ 'K.*'"))
for item in person_list:
    print(item.name)

Kevin Bacon
Kiefer Sutherland
Kevin Pollak
Kelly McGillis
Keanu Reeves
Kelly Preston


### One example:
<id>:
493841
countries:
China
country_codes:
CHN
icij_id:
58287E0FD37852000D9D5AB8B27A2581
name:
Tian Yuan
node_id:
12000002
sourceID:
Panama Papers
valid_until:
The Panama Papers data is current through 2015
### relationship:
registered_address
shareholder_of
intermediary_of

"elements": {
        "nodes": [
          {"data": {"id": "172", "name": "Tom Cruise", "label": "Person"}},
          {"data": {"id": "183", "title": "Top Gun", "label": "Movie"}}
        ],
        "edges": [{"data": {"source": "172", "target": "183", "relationship": "Acted_In"}}]
      }

In [80]:
for share_of in tian.shareholder_of:
    print(officer_of.name)


CELLO HOLDINGS LIMITED
